In [5]:
from config import load_dictionaries, init_language_tool_pl, init_language_tool_en

from typing import List
from tqdm import tqdm

from dao.attribute import DAOAttributePL

from models.attribute import AttributePLInDB
from analysis.attribute_retriving import spelling_and_grammar_check, dictionary_check

In [2]:
load_dictionaries()
init_language_tool_pl()
init_language_tool_en()
dao_attributes = DAOAttributePL(collection_name="attributes-24-12-16-recalc-24-12-22.1-pgryka")
not_processed_word_dict: List[AttributePLInDB] = dao_attributes.find_many_by_query({'number_of_unrecognized_words_lang_tool': None})

In [6]:
init_language_tool_en()

In [7]:
for attribute_in_db in tqdm(not_processed_word_dict, total=len(not_processed_word_dict),
                            desc=f'Calculating word dicts statistics of lab reports', unit='Lab reports', miniters=1):
    text_to_analyse = attribute_in_db.stylometrix_metrics.text
    text_errors_by_category, number_of_errors, number_of_abbreviations, number_of_unrecognized_words = spelling_and_grammar_check(text_to_analyse, 'pl')
    number_of_unrecognized_words_dict_check = dictionary_check(text_to_analyse)

    dao_attributes.update_one({'_id': attribute_in_db.id},
                              {"$set": {'number_of_unrecognized_words_lang_tool': number_of_unrecognized_words,
                                        'number_of_abbreviations_lang_tool': number_of_abbreviations,
                                        'number_of_unrecognized_words_dict_check': number_of_unrecognized_words_dict_check}})

Calculating word dicts statistics of lab reports: 100%|██████████| 4756/4756 [1:17:51<00:00,  1.02Lab reports/s]  
